# Installing libraries

In [1]:
!pip install langchain langchain-google-genai langchain_community pypdf faiss-cpu


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


Adding API Keys

In [2]:
import os
import getpass
google_api_path = 'google_api_path.txt'
if "GOOGLE_API_KEY" not in os.environ:
    try:
        with open(google_api_path, "r") as file:
            api_key = file.read().strip()
            os.environ['GOOGLE_API_KEY'] = api_key
    except:
        os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

# Importing libraries

In [3]:
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.prompts import load_prompt
# from langchain_text_splitters.base import TextSplitter
# from pymongo import MongoClient
# from pymongo.mongo_client import MongoClient
# from pymongo.server_api import ServerApi
import json
import time
import datetime
import pypdf

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Operations on data

In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=400,
    chunk_overlap=40
)

In [5]:
!curl -o servicenow-docs.pdf https://servicenow-docs.readthedocs.io/_/downloads/en/latest/pdf/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  299k  100  299k    0     0  5351k      0 --:--:-- --:--:-- --:--:-- 5351k


In [6]:
pdf_source = 'servicenow-docs.pdf'
pdf_loader = PyPDFLoader(pdf_source)
pdf_data = pdf_loader.load_and_split(text_splitter=text_splitter)

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectors = FAISS.from_documents(pdf_data, embeddings)
vectors.save_local("faiss_index")

In [16]:
pdf_data

[Document(metadata={'source': 'servicenow-docs.pdf', 'page': 0, 'page_label': '1'}, page_content='ServiceNow Documentation\nDocumentation\nRelease 0.01\nMark Carter\nNov 13, 2017'),
 Document(metadata={'source': 'servicenow-docs.pdf', 'page': 2, 'page_label': 'i'}, page_content='Contents\n1 Copying Data 3\n1.1 Worknotes . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3\n1.2 Currency Fields . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3\n2 CORS Error 5\n2.1 How to get around CORS Exception . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n3 Date Time Servicenow 7'),
 Document(metadata={'source': 'servicenow-docs.pdf', 'page': 2, 'page_label': 'i'}, page_content='3 Date Time Servicenow 7\n3.1 Update Review Date . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\n3.2 Comparing Due Date with Start Date . . . . . . . . . . . . . . . . . . .

# Calling LLM and data

In [17]:
retriever = vectors.as_retriever(search_type="similarity", search_kwargs={"k":6, "include_metadata":True, "score_threshold":0.9})
chain = ConversationalRetrievalChain.from_llm(llm=ChatGoogleGenerativeAI(temperature=0.0,model='gemini-1.5-pro-002'), 
                                                retriever=retriever,return_source_documents=True,verbose=True,chain_type="stuff",
                                                max_tokens_limit=4097)

In [18]:
chat_history = []
result = chain({"system": 
        "You are a ServiceNow expert and answer based on the document provided. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.", 
                        "question": " How to get around CORS Exception",
                        "chat_history":chat_history})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
CHAPTER 2
CORS Error
2.1 How to get around CORS Exception
chrome.exe --user-data-dir="C:/Chrome dev session" --disable-web-security
Depending on location of Chrome, ﬁle can be run with single line below
"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" --user-data-dir="C:/
˓→Chrome dev session" --disable-web-security
5

ServiceNow Documentation Documentation, Release 0.01
6 Chapter 2. CORS Error

why it would fail anywhere else.
var request = new sn_ws.RESTMessageV2();
request.setEndpoint('https://dev22614.service-now.com/api/now/table/u_cart_order');
request.setHttpMethod('POST');
31

Contents
1 Copying Data 3
1.1 Worknotes . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [19]:
result

{'system': 'You are a ServiceNow expert and answer based on the document provided. Be polite and provide answers based on the provided context only. Use only the provided data and not prior knowledge.',
 'question': ' How to get around CORS Exception',
 'chat_history': [],
 'answer': 'Run Chrome with the following flags:\n\n```\nchrome.exe --user-data-dir="C:/Chrome dev session" --disable-web-security \n```\n\nOr, depending on the location of Chrome:\n\n```\n"C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe" --user-data-dir="C:/Chrome dev session" --disable-web-security\n```\n\n**Warning:** Be careful of security implications when disabling web security.',
 'source_documents': [Document(id='f82f5b9f-8fda-4c62-9d6d-efd3b0012a37', metadata={'source': 'servicenow-docs.pdf', 'page': 8, 'page_label': '5'}, page_content='CHAPTER 2\nCORS Error\n2.1 How to get around CORS Exception\nchrome.exe --user-data-dir="C:/Chrome dev session" --disable-web-security\nDepending on location

In [20]:
print(result['answer'])

Run Chrome with the following flags:

```
chrome.exe --user-data-dir="C:/Chrome dev session" --disable-web-security 
```

Or, depending on the location of Chrome:

```
"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" --user-data-dir="C:/Chrome dev session" --disable-web-security
```

**Warning:** Be careful of security implications when disabling web security.


In [21]:
print(result['source_documents'])

[Document(id='f82f5b9f-8fda-4c62-9d6d-efd3b0012a37', metadata={'source': 'servicenow-docs.pdf', 'page': 8, 'page_label': '5'}, page_content='CHAPTER 2\nCORS Error\n2.1 How to get around CORS Exception\nchrome.exe --user-data-dir="C:/Chrome dev session" --disable-web-security\nDepending on location of Chrome, ﬁle can be run with single line below\n"C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe" --user-data-dir="C:/\n˓→Chrome dev session" --disable-web-security\nWarning: Goes without saying !! be careful of security\n5'), Document(id='10127c63-e964-409e-bc3c-7c6e0b13ab42', metadata={'source': 'servicenow-docs.pdf', 'page': 9, 'page_label': '6'}, page_content='ServiceNow Documentation Documentation, Release 0.01\n6 Chapter 2. CORS Error'), Document(id='aecf72c8-f222-43e5-b197-2b5129382704', metadata={'source': 'servicenow-docs.pdf', 'page': 34, 'page_label': '31'}, page_content="why it would fail anywhere else.\nvar request = new sn_ws.RESTMessageV2();\nrequest.setEndpo

In [22]:
print(result['source_documents'][0])

page_content='CHAPTER 2
CORS Error
2.1 How to get around CORS Exception
chrome.exe --user-data-dir="C:/Chrome dev session" --disable-web-security
Depending on location of Chrome, ﬁle can be run with single line below
"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" --user-data-dir="C:/
˓→Chrome dev session" --disable-web-security
5' metadata={'source': 'servicenow-docs.pdf', 'page': 8, 'page_label': '5'}


In [14]:
for i in result['source_documents']:
    print(i)
    print("-----")

page_content='CHAPTER 2
CORS Error
2.1 How to get around CORS Exception
chrome.exe --user-data-dir="C:/Chrome dev session" --disable-web-security
Depending on location of Chrome, ﬁle can be run with single line below
"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" --user-data-dir="C:/
˓→Chrome dev session" --disable-web-security
5' metadata={'source': 'servicenow-docs.pdf', 'page': 8, 'page_label': '5'}
-----
page_content='ServiceNow Documentation Documentation, Release 0.01
6 Chapter 2. CORS Error' metadata={'source': 'servicenow-docs.pdf', 'page': 9, 'page_label': '6'}
-----
page_content='why it would fail anywhere else.
var request = new sn_ws.RESTMessageV2();
request.setEndpoint('https://dev22614.service-now.com/api/now/table/u_cart_order');
request.setHttpMethod('POST');
31' metadata={'source': 'servicenow-docs.pdf', 'page': 34, 'page_label': '31'}
-----
page_content='Contents
1 Copying Data 3
1.1 Worknotes . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 